<a href="https://colab.research.google.com/github/talhaanwarch/Sarcasm-nlp/blob/master/twitter_multi_input_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub
!pip install tensorflow-addons
#!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 49.2MB/s 
     |████████████████████████████████| 3.9MB 51.3MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 1.0MB 2.8MB/s 


In [3]:
cd /content/drive/My\ Drive/dataset/sarcasm

/content/drive/My Drive/dataset/sarcasm


In [4]:
ls

model_gru.png
__pycache__/
sarcasm_detection_shared_task_reddit_testing.jsonl
sarcasm_detection_shared_task_reddit_training.jsonl
sarcasm_detection_shared_task_twitter_testing.jsonl
sarcasm_detection_shared_task_twitter_training.jsonl
tokenization.py


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score


In [0]:
twitter_train=pd.read_json('sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)

In [7]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')


In [9]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,[A minor child deserves privacy and should be ...
1,SARCASM,trying to protest about talking about him a...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,he makes an insane about of money from the ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,meanwhile trump wont even release his sat sc...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')


In [11]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [12]:
len(twitter_train)

5000

In [13]:
from collections import Counter
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(twitter_train['label'])

In [15]:
twitter_test=pd.read_json('sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [0]:
twitter_test['response']=twitter_test['response'].str.replace('@USER', "") 
twitter_test['response']=twitter_test['response'].str.replace('\d+', '')
twitter_test['response']=twitter_test['response'].str.lower()
twitter_test['response']=twitter_test['response'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].apply(lambda x: ','.join(map(str, x)))
twitter_test['context']=twitter_test['context'].str.replace('@USER', "") 
twitter_test['context']=twitter_test['context'].str.lower()
twitter_test['context']=twitter_test['context'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].str.replace('\d+', '')


In [17]:
twitter_test.head()

,context,response,id
0,well now that s problematic af url my year ...,my year old that just finished reading ni...,twitter_1
1,last week the fake news said that a section of...,how many verifiable lies has he told now d...,twitter_2
2,let s aplaud brett when he deserves it he co...,maybe docs just a scrub of a coach i mean ...,twitter_3
3,women generally hate this president whats up ...,is just a cover up for the real hate inside ...,twitter_4
4,dear media remoaners you excitedly sharing cl...,the irony being that he even has to ask why,twitter_5


#BERT

In [0]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
train_response = bert_encode(twitter_train.response, tokenizer, max_len=130)
train_context = bert_encode(twitter_train.context, tokenizer, max_len=130)
train_labels=twitter_train.label.values
#test_input = bert_encode(test_bert.values, tokenizer, max_len=160)

In [0]:
test_response = bert_encode(twitter_test.response, tokenizer, max_len=130)
test_context = bert_encode(twitter_test.context, tokenizer, max_len=130)

In [0]:
test_gen=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

In [0]:
 max_len=130

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa

In [0]:
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)

# ...

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

In [0]:
contex_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_word_ids")
contex_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_mask")
contex_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_segment_ids")
_, context_sequence_output = bert_layer([contex_input_word_ids, contex_input_mask, contex_segment_ids])
context_clf_output = context_sequence_output[:, 0, :]


reponse_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_word_ids")
reponse_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_mask")
reponse_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_segment_ids")
_, reponset_sequence_output = bert_layer([reponse_input_word_ids, reponse_input_mask, reponse_segment_ids])
reponset_clf_output = reponset_sequence_output[:, 0, :]



conc=concatenate([context_clf_output,reponset_clf_output])
conc=Dropout(0.3)(conc)
out = Dense(1, activation='sigmoid')(conc)
    
model = Model(inputs=[contex_input_word_ids, contex_input_mask, contex_segment_ids,reponse_input_word_ids,reponse_input_mask,reponse_segment_ids], outputs=out)
model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    


In [0]:
train_gen=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

In [0]:
train_history = model.fit(
    train_gen, train_label,
    epochs=10,
    batch_size=8)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 1683s 337ms/sample - loss: 0.6025 - accuracy: 0.6674
Epoch 2/10
5000/5000 [==============================] - 1632s 326ms/sample - loss: 0.4808 - accuracy: 0.7584
Epoch 3/10
 784/5000 [===>..........................] - ETA: 22:55 - loss: 0.4522 - accuracy: 0.7819

In [0]:
# len(train_gen),len(train_label)

In [0]:
# train_context[0].shape

In [0]:
# import tensorflow_addons as tfa
# from sklearn.model_selection import StratifiedKFold


In [0]:
# def unpack(a, b, c, d,e,f,g): 
#     #print(a[g].shape, b[g].shape, c[g].shape, d[g].shape,e[g].shape,f[g].shape) 
#     return [a[g], b[g], c[g], d[g],e[g],f[g]]

In [0]:
#unpack(*train_gen,train)

In [0]:
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=5, shuffle=False)
# scores = []
# test=[]
# s=0
# for train, val in kfold.split(list(zip(*train_gen)), train_label):
#   model.fit(unpack(*train_gen,train),train_label[train],batch_size=6,epochs=5,verbose=2,
#             validation_data=(unpack(*train_gen,val), train_label[val]))
   

#   y_pred = model.predict(unpack(*train_gen,val), batch_size=6, verbose=1)

#   test.append(model.predict(test_gen, batch_size=6, verbose=1).ravel())

#   y_pred = (y_pred > 0.5)
#   f1=f1_score(train_label[val], y_pred, average='macro')
#   s+=1
#   print(s,' f1 score  is ',f1)
#   scores.append(f1)
#   tf.keras.backend.clear_session()

In [0]:
test=model.predict(test_gen, batch_size=6, verbose=1).ravel()

In [0]:
test.shape

In [0]:

sub=test.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([twitter_test['id'], sub], axis=1)
sub.head()
sub.to_csv('/content/answer.txt',sep=',',index=False,header=None)

In [0]:
test[0].shape